# # HuggingFace로 두 문장의 논리적 모순 분류하기


In [2]:
!pip -q install transformers datasets evaluate accelerate scikit-learn sacrebleu sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00


In [3]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import (
  EarlyStoppingCallback,
  AutoTokenizer,
  AutoModelForSequenceClassification,
  BertConfig,
  TrainingArguments,
  Trainer,
)


mnli = load_dataset("nyu-mll/glue", "mnli")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["premise"], data["hypothesis"], truncation=True)

mnli_tokenized = mnli.map(preprocess_function, batched=True)

mnli_split = mnli_tokenized['train'].train_test_split(test_size=0.2)
mnli_train, mnli_val = mnli_split['train'], mnli_split['test']
mnli_test = mnli_tokenized['validation_matched']


config = BertConfig(
  hidden_size=128,
  intermediate_size=128,
  num_hidden_layers=4,
  num_attention_heads=8,
  num_labels=len(mnli['train'].unique('label')),
  hidden_dropout_prob=0.2,
  attention_probs_dropout_prob=0.2,
)


model = AutoModelForSequenceClassification.from_config(config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [4]:
training_args = TrainingArguments(
    output_dir='./temp',
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_strategy="epoch",
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    load_best_model_at_end=True,
)

In [5]:
accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mnli_train,
    eval_dataset=mnli_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [6]:
trainer.train()

TrainOutput(global_step=49090, training_loss=0.9978734550856845, metrics={'train_runtime': 1386.5848, 'train_samples_per_second': 2265.718, 'train_steps_per_second': 35.404, 'total_flos': 861666604502130.0, 'train_loss': 0.9978734550856845, 'epoch': 10.0})

In [11]:
from transformers import pipeline
trainer.evaluate(mnli_test)


{'eval_loss': 0.9852012395858765,
 'eval_accuracy': 0.5162506367804381,
 'eval_runtime': 2.2991,
 'eval_samples_per_second': 4269.046,
 'eval_steps_per_second': 66.982,
 'epoch': 10.0}